## convert using docker 
https://github.com/NVIDIA/TensorRT-LLM/tree/main/examples/bert

### tensorrt-llm version 0.12.0

1. docker run -it --rm --network host -v ./:/data --gpus "device=1" trtllm_tritonserver bash
2. pip install pycuda tensorflow h5py==3.10.0 'transformers<=4.42.4'
3. cd /data/third_party/TensorRT-LLM/examples/bert
4. code convert

NOTES:
1. comment these following line:
    - line 129 /usr/local/lib/python3.10/dist-packages/tensorrt_llm/profiler.py
    ```
    Comment
        if pynvml.__version__ < '11.5.0' or driver_version < '526':
            logger.warning(
                f'Found pynvml=={pynvml.__version__} and cuda driver version '
                f'{driver_version}. Please use pynvml>=11.5.0 and cuda '
                f'driver>=526 to get accurate memory usage.')
            # Support legacy pynvml. Note that an old API could return
            # wrong GPU memory usage.
            _device_get_memory_info_fn = pynvml.nvmlDeviceGetMemoryInfo
        else:
    ```
    => Only reserve after `else`, then fix index and save file


##### custom mask

In [ ]:
# error:
[02/02/2025-04:25:43] [TRT] [I] Global timing cache in use. Profiling results in this builder pass will be stored.
[02/02/2025-04:25:43] [TRT] [E] IBuilder::buildSerializedNetwork: Error Code 4: Internal Error (kOPT values for profile 0 violate shape constraints: BertRetriever/layers/0/attention/__add___L321/elementwise_binary_L2855/ELEMENTWISE_SUM_0: dimensions not compatible for elementwise. Broadcast has incompatible dimensions: 128 != 256 && 128 != 1 && 256 != 1.)
Traceback (most recent call last):
  File "/data/third_party/TensorRT-LLM/examples/bert/build_retrieve.py", line 273, in <module>
    engine = builder.build_engine(network, builder_config)
  File "/usr/local/lib/python3.10/dist-packages/tensorrt_llm/_common.py", line 204, in decorated
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorrt_llm/builder.py", line 411, in build_engine
    assert engine is not None, 'Engine building failed, please check the error log.'
AssertionError: Engine building failed, please check the error log.

##### fp32

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint"

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint"

In [14]:
# add new file for customizing input 
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint"

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint"

SyntaxError: invalid syntax (206555486.py, line 3)

##### fp16

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/bert-retrieve-qry-base_float16_tllm_checkpoint"

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint"

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint"

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint"


##### fp32 + bert_attetion_plugin float32

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

##### fp16 + bert_attetion_plugin float16

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

### tensorrt_llm v0.16.0

1. docker run -it --rm --network host -v ./:/data --gpus "device=1" heronq02/trtllm-tritonserver:24.12 bash
2. pip install pycuda tensorflow h5py==3.10.0 'transformers<=4.42.4'
3. cd /data/third_party/TensorRT-LLM/examples/bert
4. code convert

##### fp32

In [ ]:
# v0.16

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-qry-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1

trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=disable \
    --bert_attention_plugin=float32 \
    --max_batch_size 1 \
    --max_input_len 512

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/home/tiennv/hungnq/rtvserving/mbert-retrieve-qry-base"   \
    --output_dir "/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1

trtllm-build --checkpoint_dir "/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --output_dir="/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=disable \
    --bert_attention_plugin=float32 \
    --max_batch_size 1 \
    --max_input_len 512



python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-ctx-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1


trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=disable \
    --bert_attention_plugin=float32 \
    --max_batch_size 10 \
    --max_input_len 512

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/home/tiennv/hungnq/rtvserving/mbert-retrieve-ctx-base"   \
    --output_dir "/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1

trtllm-build --checkpoint_dir "/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --output_dir="/home/tiennv/hungnq/rtvserving/outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=disable \
    --bert_attention_plugin=float32 \
    --max_batch_size 10 \
    --max_input_len 512
    




##### fp16

In [ ]:
# v0.16

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-qry-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2"  \
    --dtype float16  \
    --tp_size 2

trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float16 \
    --max_batch_size 10 \
    --max_input_len 512

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-ctx-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2"  \
    --dtype float16  \
    --tp_size 2


trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float16 \
    --max_batch_size 8 \
    --max_input_len 512


## Evaluate

In [1]:
import os
os.environ['CUDA_DEVICE'] = '0'

### utils

In [2]:
from torch import nn
import torch

cross_entropy = nn.CrossEntropyLoss(reduction='mean')

def compute_loss(scores, target):
    return cross_entropy(scores, target)

def compute_similarity(q_reps, p_reps):
    if not isinstance(q_reps, torch.Tensor):
        q_reps = torch.tensor(q_reps)
    if not isinstance(p_reps, torch.Tensor):
        p_reps = torch.tensor(p_reps)
    return torch.matmul(q_reps, p_reps.transpose(0,1))

In [3]:
import argparse
import json
import os

# isort: off
import torch
import tensorrt as trt
# isort: on

import tensorrt_llm
from tensorrt_llm import logger
from tensorrt_llm.runtime import Session, TensorInfo

# from build import get_engine_name  # isort:skip
def get_engine_name(model, dtype, tp_size, rank):
    return '{}_{}_tp{}_rank{}.plan'.format(model, dtype, tp_size, rank)

def trt_dtype_to_torch(dtype):
    if dtype == trt.float16:
        return torch.float16
    elif dtype == trt.float32:
        return torch.float32
    elif dtype == trt.int32:
        return torch.int32
    else:
        raise TypeError("%s is not supported" % dtype)

/home/tiennv/.conda/envs/trt-hung-10_7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-13 19:56:39.630156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739451399.642938 2167575 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739451399.646919 2167575 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 19:56:39.660610: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performanc

[TensorRT-LLM] TensorRT-LLM version: 0.16.0


In [4]:
import torch
import tensorrt_llm

tensorrt_llm.logger.set_level("info")

def get_model_config(config_path):
    
    config_path = os.path.join(config_path, 'config.json')

    with open(config_path, 'r') as f:
        config = json.load(f)

    # V0.16.0 add builder 
    # assert config["plugin_config"]["remove_input_padding"] == False, \\
    # assert config['build_config']["plugin_config"]["remove_input_padding"] == False, \
    #     "Please refer to run_remove_input_padding.py for running BERT models with remove_input_padding enabled"
    return config

def get_session(config_path):
    config = get_model_config(config_path)
    # world_size = config['builder_config']['tensor_parallel']
    world_size = config['build_config']['auto_parallel_config']['world_size']
    assert world_size == tensorrt_llm.mpi_world_size(), \
        f'Engine world size ({world_size}) != Runtime world size ({tensorrt_llm.mpi_world_size()})'

    runtime_rank = tensorrt_llm.mpi_rank() if world_size > 1 else 0
    runtime_mapping = tensorrt_llm.Mapping(world_size,
                                            runtime_rank,
                                            tp_size=world_size)
    
    torch.cuda.set_device(runtime_rank % runtime_mapping.gpus_per_node)
    # dtype = config['builder_config']['precision']
    # model_name = config['builder_config']['name']
    # serialize_path = get_engine_name(
    #     model_name, 
    #     dtype, world_size, runtime_rank
    # )
    serialize_path = 'rank0.engine'
    serialize_path = os.path.join(config_path, serialize_path)
    print(serialize_path)

    logger.info(f'Loading engine from {serialize_path}')
    with open(serialize_path, 'rb') as f:
        engine_buffer = f.read()

    logger.info(f'Creating session from engine')
    return Session.from_serialized_engine(engine_buffer)


In [5]:
from typing import List

def process_input(input_ids_list: List[torch.Tensor],
                  token_type_ids_list: List[torch.Tensor],
                  is_roberta=False,
                  padding_idx=1):
    input_lengths = []
    position_ids_list = []
    max_input_length = 0
    for i, input_ids in enumerate(input_ids_list):
        input_len = len(input_ids)
        assert input_len == len(token_type_ids_list[i]), f"sample {i}: len(input_ids)={len(input_ids)}, " \
                                                         f"len(token_type_ids)={len(token_type_ids_list[i])}, not equal"
        input_lengths.append(input_len)
        position_ids = torch.arange(0, input_len, dtype=torch.int32)
        if is_roberta:
            position_ids = position_ids + 1 + padding_idx

        position_ids_list.append(position_ids)
        max_input_length = max(max_input_length, input_len)

    # [num_tokens]
    input_ids = torch.concat(input_ids_list).int().cuda()
    token_type_ids = torch.concat(token_type_ids_list).int().cuda()
    position_ids = torch.concat(position_ids_list).int().cuda()

    input_lengths = torch.tensor(input_lengths).int().cuda()  # [batch_size]
    max_input_length = torch.empty((max_input_length, )).int().cuda()
    return input_ids, input_lengths, token_type_ids, position_ids, max_input_length


In [6]:
from transformers import AutoTokenizer

def prepare_inputs(texts: list, tokenizer: AutoTokenizer, config) -> dict:
    #
    remove_padding = config['build_config']['plugin_config'][
        'remove_input_padding'
    ]
    model_name = config['pretrained_config']['architecture']

    # Roberta doesn't have token_type_ids, use all zeros to replace
    is_roberta = "Roberta" in model_name

    if remove_padding:
        input_ids_list = [
            torch.tensor(ids).int().cuda() \
            for ids in inputs_without_padding['input_ids']
        ]
        # attention_mask_list = inputs_without_padding['attention_mask'],
        token_type_ids_list = [
            torch.tensor(ids).int().cuda() \
            for ids in inputs_without_padding['token_type_ids']
        ]
        
        input_ids, input_lengths, token_type_ids, position_ids, max_input_length = process_input(
            input_ids_list=input_ids_list,
            token_type_ids_list=token_type_ids_list,
            is_roberta=is_roberta,
            padding_idx=config['pretrained_config']['pad_token_id']
        )

    else:

        #NOTE:Padding: pad to longest seq len
        inputs_with_padding = tokenizer(
            texts,
            padding=True,
        )

        inputs_without_padding = tokenizer(texts)

        input_ids = torch.tensor(inputs_with_padding['input_ids']).int().cuda()
        input_lengths = [len(x) for x in inputs_without_padding['input_ids']]
        input_lengths = torch.tensor(input_lengths,
                                     device=input_ids.device,
                                     dtype=torch.int32)
        
        # attention_mask = torch.tensor(inputs_with_padding['attention_mask'],
        #                               device=input_ids.device,
        #                               dtype=torch.int32)
        if is_roberta:
            token_type_ids = torch.zeros_like(torch.tensor(
                inputs_with_padding['input_ids']),
                                              device=input_ids.device,
                                              dtype=torch.int32)
        else:
            token_type_ids = torch.tensor(inputs_with_padding['token_type_ids'],
                                          device=input_ids.device,
                                          dtype=torch.int32)
            
    # NOTE: TRT-LLM perform inference
    if remove_padding:
        # NOTE: Remove padding:
        inputs = {
            "input_ids": input_ids,
            "input_lengths": input_lengths,
            "token_type_ids": token_type_ids,
            "position_ids": position_ids,
            "max_input_length": max_input_length
        }
    else:
        #NOTE: Padding:
        inputs = {
            'input_ids': input_ids,
            'input_lengths': input_lengths,
            'token_type_ids': token_type_ids,
        }
    return inputs

            

In [7]:
import time

def encode_trtllm(texts, model, tokenizer, config, stream):
    # check if tokenize length is min 128
    inputs_model = prepare_inputs(texts, tokenizer, config)

    if config['build_config']['plugin_config']['remove_input_padding']:
        output_info = model.infer_shapes([
            TensorInfo("input_ids", trt.DataType.INT32, inputs_model['input_ids'].shape),
            TensorInfo("input_lengths", trt.DataType.INT32, inputs_model['input_lengths'].shape),
            TensorInfo("token_type_ids", trt.DataType.INT32, inputs_model['token_type_ids'].shape),
            TensorInfo("position_ids", trt.DataType.INT32, inputs_model['position_ids'].shape),
            TensorInfo("max_input_length", trt.DataType.INT32, inputs_model['max_input_length'].shape)
        ])
    else:
        #NOTE: Padding:
        output_info = model.infer_shapes([
            TensorInfo("input_ids", trt.DataType.INT32, inputs_model['input_ids'].shape),
            TensorInfo("input_lengths", trt.DataType.INT32, inputs_model['input_lengths'].shape),
            TensorInfo("token_type_ids", trt.DataType.INT32, inputs_model['token_type_ids'].shape),
        ])
    
    outputs = {
        t.name: torch.empty(
            tuple(t.shape),
            dtype=trt_dtype_to_torch(t.dtype),
            device='cuda'
        )
        for t in output_info
    }

    start_time = time.time()
    ok = model.run(inputs_model, outputs, stream)
    assert ok, "Runtime execution failed"
    torch.cuda.synchronize()
    end_time = time.time() - start_time

    embeddings = outputs['hidden_states']
    return embeddings.reshape(len(texts), -1, 768)[:, 0], end_time

In [8]:
from tqdm import tqdm
import torch
import time
from typing import Callable
import inspect

def eval_accuracy_trt(
    data, 
    encode_fn = Callable, 
    num_passages=65, 
    qry_model_dir=None, 
    ctx_model_dir=None, 
    tokenizer_query=None,
    tokenizer_ctx=None, 
    device='cpu',
):

    # assert model_ctx is not None, "model_ctx is required"
    # assert model_qry is not None, "model_qry is required"
    assert tokenizer_ctx is not None, "tokenizer_ctx is required"
    assert tokenizer_query is not None, "tokenizer_query is required"
    assert 'query' in data.column_names, "data must have query column"
    assert 'positive' in data.column_names, "data must have positive column"
    assert 'negatives' in data.column_names, "data must have negatives column"
    # len of arguemtn of encode_fn must be 4
    # print(inspect.getargspec(encode_fn).args)
    assert len(inspect.getargspec(encode_fn).args) == 5, "encode_fn must have 5 arguments"

    accuracy = 0

    model_qry = get_session(qry_model_dir)
    model_ctx = get_session(ctx_model_dir)

    if device != "cpu":
        model_ctx = model_ctx.to(device)
        model_qry = model_qry.to(device)

    time_query_total = 0
    time_query_run = 0
    time_passage_total = 0
    time_passage_run = 0

    for i in tqdm(range(len(data))):

        start_time = time.time()
        query_config = get_model_config(qry_model_dir)
        stream = torch.cuda.current_stream().cuda_stream
        # print(model_qry)
        # print(tokenizer_query)
        # print(query_config)

        #! CHANGE HERE
        query_batch = [data[i]['query']]
        query, time_query = encode_fn(query_batch, model_qry, tokenizer_query, config=query_config, stream=stream)
        end_time = time.time() - start_time
        time_query_total += end_time
        time_query_run += time_query

        # concate 10 passages
        passage_config = get_model_config(ctx_model_dir)
        concate_passage = [data[i]['positive']] + data[i]['negatives'][:num_passages-1]
        start_time = time.time()
        #! CHANGE HERE
        # print(model_ctx)
        # print(tokenizer_ctx)
        # print(passage_config)
        
        stream2 = torch.cuda.current_stream().cuda_stream
        encoded_passages, time_ctx = encode_fn(concate_passage, model_ctx, tokenizer_ctx, config=passage_config, stream=stream2)
        end_time = time.time() - start_time
        time_passage_total += end_time
        time_passage_run += time_ctx

        # accuracy
        scores = compute_similarity(query, encoded_passages)
        # print(scores)
        if scores.argmax(dim=1).detach().cpu().numpy() != 0:
            continue
        accuracy += 1

    return accuracy / len(data), time_query_run/ len(data), time_passage_run/ len(data), time_query_total/ len(data), time_passage_total/ len(data)

### prepare datasets

In [9]:
import datasets
from datasets import concatenate_datasets
en_eval = datasets.load_dataset('tiennv/mmarco-passage-vi', split='train[-500:]', cache_dir="../datahub/")
vi_eval = datasets.load_dataset('tiennv/mmarco-passage-vi', split='train[-500:]', cache_dir="../datahub/")

dataset_eval = concatenate_datasets([en_eval, vi_eval])
dataset_eval

Using the latest cached version of the dataset since tiennv/mmarco-passage-vi couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ../datahub/tiennv___mmarco-passage-vi/default/0.0.0/5ee2171bc2bc0880d2f35c16063096ec1c4dc4da (last modified on Sun Jan 26 14:00:08 2025).
Using the latest cached version of the dataset since tiennv/mmarco-passage-vi couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ../datahub/tiennv___mmarco-passage-vi/default/0.0.0/5ee2171bc2bc0880d2f35c16063096ec1c4dc4da (last modified on Sun Jan 26 14:00:08 2025).


Dataset({
    features: ['query_id', 'query', 'positive_id', 'positive', 'negatives'],
    num_rows: 1000
})

### run with attention plugin
use code run of tensorrtllm

In [10]:
qry_engine_dir = "../outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"
ctx_engine_dir = "../outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"

In [11]:
from transformers import AutoTokenizer
accuracy, time_query_run, time_passage_run, time_query_total, time_passage_total = eval_accuracy_trt(
    dataset_eval, 
    encode_trtllm,
    num_passages=10, 
    qry_model_dir=qry_engine_dir, 
    ctx_model_dir=ctx_engine_dir, 
    tokenizer_query=AutoTokenizer.from_pretrained("../mbert-retrieve-qry-base/"),
    tokenizer_ctx=AutoTokenizer.from_pretrained("../mbert-retrieve-ctx-base/"),
    device="cpu"
)
print(f"Accuracy: {accuracy}")
print(f"Time Query Run: {time_query_run}")  
print(f"Time Passage Run: {time_passage_run}")
print(f"Time Query Total: {time_query_total}")
print(f"Time Passage Total: {time_passage_total}")

../outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine
[02/13/2025-19:56:42] [TRT-LLM] [I] Loading engine from ../outputs/trtllm_2/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine


/tmp/ipykernel_2167575/2073153388.py:27: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  assert len(inspect.getargspec(encode_fn).args) == 5, "encode_fn must have 5 arguments"


[02/13/2025-19:56:43] [TRT-LLM] [I] Creating session from engine
[02/13/2025-19:56:43] [TRT] [I] Loaded engine size: 677 MiB
[02/13/2025-19:56:43] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +76, now: CPU 0, GPU 752 (MiB)
../outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2/rank0.engine
[02/13/2025-19:56:43] [TRT-LLM] [I] Loading engine from ../outputs/trtllm_2/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2/rank0.engine
[02/13/2025-19:56:43] [TRT-LLM] [I] Creating session from engine
[02/13/2025-19:56:43] [TRT] [I] Loaded engine size: 678 MiB
[02/13/2025-19:56:43] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +478, now: CPU 0, GPU 1906 (MiB)


  0%|          | 2/1000 [00:00<01:08, 14.54it/s]

[02/13/2025-19:56:43] [TRT] [W] Using default stream in enqueueV3() may lead to performance issues due to additional calls to cudaStreamSynchronize() by TensorRT to ensure correct synchronization. Please use non-default stream instead.
[02/13/2025-19:56:44] [TRT] [W] Using default stream in enqueueV3() may lead to performance issues due to additional calls to cudaStreamSynchronize() by TensorRT to ensure correct synchronization. Please use non-default stream instead.


100%|██████████| 1000/1000 [00:35<00:00, 28.14it/s]

Accuracy: 1.0
Time Query Run: 0.003019651412963867
Time Passage Run: 0.029099109888076784
Time Query Total: 0.0037817022800445558
Time Passage Total: 0.030956610918045045


## Appendix

combine calibration (API settings dfefault)

STATUS: 
- [Not supported model](https://github.com/NVIDIA/TensorRT-LLM/issues/1614#issuecomment-2122086630) 
- Config model type không có Bert - [line 110](https://github.com/NVIDIA/TensorRT-LLM/blob/main/tensorrt_llm/quantization/quantize_by_modelopt.py#L550)


### prepare

In [ ]:
import datasets
from datasets import concatenate_datasets 

number_samples = 250 
en = datasets.load_dataset('tiennv/mmarco-passage-vi', split=f'train[:{number_samples}]',
                          cache_dir="./datahub")
vi = datasets.load_dataset('tiennv/mmarco-passage-vi', split=f'train[:{number_samples}]', cache_dir="./datahub")

dataset_calib = concatenate_datasets([en, vi])
dataset_calib

### calib API

In [ ]:
from transformers import DataCollatorWithPadding, AutoTokenizer

query_tokenizer = AutoTokenizer.from_pretrained('mbert-retrieve-qry-base/')
ctx_tokenizer = AutoTokenizer.from_pretrained('mbert-retrieve-ctx-base/')

def query_collate_fn(examples):
    query = [example['query'] for example in examples]
    encoded_input = query_tokenizer(
        query, 
        padding='max_length', 
        truncation=True, 
        max_length=512, 
        return_tensors='pt'
    )
    return encoded_input


def ctx_collate_fn(examples):

    concate_passage = []
    for example in examples:
        concate_passage.extend(
            [example['positive']] + example['negatives'][:9]
        )

    # concate_passage = [examples['positive']] + examples['negatives'][:9]
    encoded_input = ctx_tokenizer(
        concate_passage, 
        padding='max_length', 
        truncation=True, 
        max_length=512, 
        return_tensors='pt',
        return_attention_mask=False
    )
    return encoded_input


In [ ]:
# Prepare the calibration set and define a forward loop
import torch

from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

batch_size = 4
calib_batches = number_samples*2 // batch_size

num_workers = 4
device = "cuda:0" if torch.cuda.is_available() else "cpu" #! in docker with gpus specify, device_id is 0

calib_query_loader = torch.utils.data.DataLoader(
    dataset_calib, 
    batch_size=batch_size,
    collate_fn=query_collate_fn,
    num_workers=num_workers, 
    pin_memory=True
)

calib_ctx_loader = torch.utils.data.DataLoader(
    dataset_calib, 
    batch_size=batch_size,
    collate_fn=ctx_collate_fn,
    num_workers=num_workers, 
    pin_memory=True
)


query_model = AutoModel.from_pretrained('mbert-retrieve-qry-base/', return_dict=True)
ctx_model = AutoModel.from_pretrained('mbert-retrieve-ctx-base/', return_dict=True)

query_model.to(device)
ctx_model.to(device)
print("Initialize ...")

In [ ]:
from tqdm import tqdm
def calibrate_loop_query():
    for i, (encode_input) in tqdm(enumerate(calib_query_loader), total=calib_batches):
        for k, v in encode_input.items():
            encode_input[k] = v.to(device)
            # print(k, v.shape)
        query_model(**encode_input)
        if i >= calib_batches:
            break

def calibrate_loop_ctx():
    for i, (encode_input) in tqdm(enumerate(calib_ctx_loader), total=calib_batches):
        for k, v in encode_input.items():
            encode_input[k] = v.to(device)
            # print(k, v.shape)
        ctx_model(**encode_input)
        if i >= calib_batches:
            break

### quantize

In [ ]:
import modelopt.torch.quantization as atq
# Select the quantization config, for example, FP8
config = atq.FP8_DEFAULT_CFG
# PTQ with in-place replacement to quantized modules
with torch.no_grad():
    atq.quantize(query_model, config, forward_loop=calibrate_loop_query)

In [ ]:
from modelopt.torch.export import export_tensorrt_llm_checkpoint

decoder_type="bert"
dtype=torch.float32
export_dir="./mbert-retrieve-qry-base-quantize-trtllm-fp8"
# ["fp8", "int8_sq", "int4_awq", "w4a8_awq", "int8_wo", "int4_wo", "full_prec"]
qformat="fp8"
DEFAULT_MAX_SEQ_LEN=512
tp_size=1
pp_size=1
awq_block_size=128
# ["int8", "fp8", None]
kv_cache_dtype="int8"


with torch.inference_mode():
    export_tensorrt_llm_checkpoint(
        model=query_model,  # The quantized model.
        decoder_type="bert",
        # decoder_type,  # The type of the model as str, e.g gptj, llama or gptnext.
        dtype=dtype,  # The exported weights data type as torch.dtype.
        export_dir=export_dir,  # The directory where the exported files will be stored.
        inference_tensor_parallel=tp_size,  # The tensor parallelism size for inference.
        inference_pipeline_parallel=pp_size,  # The pipeline parallelism size for inference.
    )

In [ ]:
# Select the quantization config, for example, FP8
config = atq.FP8_DEFAULT_CFG
# PTQ with in-place replacement to quantized modules
with torch.no_grad():
    atq.quantize(ctx_model, config, forward_loop=calibrate_loop_ctx)

In [ ]:
from modelopt.torch.export import export_tensorrt_llm_checkpoint

decoder_type="bert"
dtype=torch.float32
export_dir="./mbert-retrieve-ctx-base-quantize-trtllm-fp8"
# ["fp8", "int8_sq", "int4_awq", "w4a8_awq", "int8_wo", "int4_wo", "full_prec"]
qformat="fp8"
DEFAULT_MAX_SEQ_LEN=512
tp_size=1
pp_size=1
awq_block_size=128
# ["int8", "fp8", None]
kv_cache_dtype="int8"

with torch.inference_mode():
    export_tensorrt_llm_checkpoint(
        model=ctx_model,  # The quantized model.
        decoder_type="bert",
        # decoder_type,  # The type of the model as str, e.g gptj, llama or gptnext.
        dtype=dtype,  # The exported weights data type as torch.dtype.
        export_dir=export_dir,  # The directory where the exported files will be stored.
        inference_tensor_parallel=tp_size,  # The tensor parallelism size for inference.
        inference_pipeline_parallel=pp_size,  # The pipeline parallelism size for inference.
    )